In [1]:
from autogen.agentchat import ConversableAgent
import autogen

config_list = autogen.config_list_from_json(
    env_or_file=r"C:\Users\sheelaj.babu\Desktop\Autogen-Training\Exercises\OAI_CONFIG_LIST.json")

llm_config = {"config_list": config_list}

c:\Users\sheelaj.babu\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''

### Create a writer agent

In [3]:
writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unleashing the Power of DeepLearning.AI

Are you ready to dive into the exciting world of artificial intelligence? Look no further than DeepLearning.AI! This groundbreaking platform is revolutionizing the way we learn and apply deep learning techniques. With its cutting-edge courses, expert instructors, and hands-on projects, DeepLearning.AI equips you with the knowledge and skills to conquer the AI landscape. From computer vision to natural language processing, this platform covers it all. Don't miss out on this opportunity to unleash the power of DeepLearning.AI – your gateway to a world of endless possibilities!


### Adding reflection

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Demystifying DeepLearning.AI: Unlocking the Magic of Artificial Intelligence in 100 Words

Blogpost:
Welcome to the world of DeepLearning.AI, where groundbreaking advancements in Artificial Intelligence (AI) become accessible to anyone with a curious mind. Join us on a journey of discovery as we unravel the magic of DeepLearning.AI in just 100 words.

Combining the power of machine learning and neural networks, DeepLearning.AI equips aspiring AI enthusiasts with the tools and knowledge to embark on their AI journey. Through expert-led courses, hands-on projects, and dynamic community support, DeepLearning.AI transforms complex concepts into understandable and actionable insights.

Whether you aspire to create innovative AI 

### Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

### Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Demystifying DeepLearning.AI: Unlocking the Magic of Artificial Intelligence in 100 Words

Blogpost:
Welcome to the world of DeepLearning.AI, where groundbreaking advancements in Artificial Intelligence (AI) become accessible to anyone with a curious mind. Join us on a journey of discovery as we unravel the magic of DeepLearning.AI in just 100 words.

Combining the power of machine learning and neural networks, DeepLearning.AI equips aspiring AI enthusiasts with the tools and knowledge to embark on their AI journey. Through expert-led courses, hands-on projects, and dynamic community support, DeepLearning.AI transforms complex concepts into understandable and actionable insights.

Whether you aspire to create innovative AI 

c:\Users\sheelaj.babu\AppData\Local\Programs\Python\Python310\lib\site-packages\autogen\agentchat\conversable_agent.py:1258: UserWarning: Cannot extract summary using reflection_with_llm: Error code: 400 - {'error': {'message': "'SEO Reviewer' does not match '^[a-zA-Z0-9_-]{1,64}$' - 'messages.1.name'", 'type': 'invalid_request_error', 'param': None, 'code': None}}. Using an empty str as summary.
  warnings.warn(


Legal Reviewer (to Critic):

Legal Reviewer:

I have reviewed the refined version of the content. Based on my legal expertise, I suggest the following:

1. Clarify the statement "DeepLearning.AI empowers anyone with an inquisitive mind." by providing specific information on who can access and benefit from DeepLearning.AI (e.g., individuals of legal age or relevant educational background).

2. Ensure that the statement "DeepLearning.AI equips learners with the tools, guidance, and a vibrant community to navigate the intricacies of AI" does not give false or exaggerated expectations of what DeepLearning.AI actually provides.

3. Confirm that the phrase "building innovative AI applications" does not imply that DeepLearning.AI guarantees success or proficiency in building such applications without proper knowledge or experience.

--------------------------------------------------------------------------------

********************************************************************************

c:\Users\sheelaj.babu\AppData\Local\Programs\Python\Python310\lib\site-packages\autogen\agentchat\conversable_agent.py:1258: UserWarning: Cannot extract summary using reflection_with_llm: Error code: 400 - {'error': {'message': "'Legal Reviewer' does not match '^[a-zA-Z0-9_-]{1,64}$' - 'messages.1.name'", 'type': 'invalid_request_error', 'param': None, 'code': None}}. Using an empty str as summary.
  warnings.warn(


Ethics Reviewer (to Critic):

I am an ethics reviewer.

Ethical Concerns:
There are no apparent ethical concerns with the content. However, I suggest the following improvement:

Suggestion:
Consider using more inclusive language to ensure that the content is accessible to a diverse audience. For example, instead of using gendered terms like "your gateway", consider using more neutral terms like "a gateway".


--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Critic (to Meta Reviewer):

Aggregrate feedback from all reviewers and give final suggestions on the writing.
Context: 




--------------------------------------------------------------------------------


c:\Users\sheelaj.babu\AppData\Local\Programs\Python\Python310\lib\site-packages\autogen\agentchat\conversable_agent.py:1258: UserWarning: Cannot extract summary using reflection_with_llm: Error code: 400 - {'error': {'message': "'Ethics Reviewer' does not match '^[a-zA-Z0-9_-]{1,64}$' - 'messages.1.name'", 'type': 'invalid_request_error', 'param': None, 'code': None}}. Using an empty str as summary.
  warnings.warn(


Meta Reviewer (to Critic):

Based on the aggregated feedback from the reviewers, it is apparent that the writing in question has both strengths and areas that need improvement. The strengths identified include strong structure and organization, detailed analysis, and a clear understanding of the topic. Reviewers agreed that these aspects contributed to a compelling and engaging piece of writing.

However, there were also areas identified as needing improvement. One common critique was the occasional lack of clarity in certain parts of the writing. Reviewers suggested that further clarifications, examples, or explanations could enhance the reader's understanding of the content. Additionally, some reviewers noted that the use of language and word choice could be refined to convey ideas more effectively and maintain consistency throughout the text.

Considering the overall feedback, it is recommended that the writer revisits the areas of concern mentioned by the reviewers. This could invo

 to get Summary:--

In [15]:
print(res.summary)

Title: DeepLearning.AI: Unleash the Power of Artificial Intelligence for All

Blogpost:
Welcome to the extraordinary realm of DeepLearning.AI, where the mysteries of Artificial Intelligence (AI) are deciphered and made accessible to everyone. Join us on a 100-word journey as we delve into the essence of DeepLearning.AI.

DeepLearning.AI empowers curious minds to explore and understand the magic behind AI. By merging machine learning and neural networks, DeepLearning.AI equips learners with the tools, guidance, and a vibrant community to navigate the intricacies of AI. Complex concepts are decoded into practical knowledge, ensuring clarity every step of the way.

Whether you're advancing your career, building innovative AI applications, or simply satisfying your curiosity, DeepLearning.AI opens doors to a world of possibilities. Let’s embark on this transformative AI journey together—welcome to the realm of DeepLearning.AI!
